# Image generation using Generative AI

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten, BatchNormalization, LeakyReLU, Input, Conv2DTranspose, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist  # You can replace with your face dataset

# Rest of the code remains the same...

# Define the generator model
def build_generator(latent_dim):
    model = Sequential()

    model.add(Dense(7 * 7 * 256, input_dim=latent_dim))
    model.add(Reshape((7, 7, 256)))
    
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(1, (7, 7), activation='sigmoid', padding='same'))

    return model

# Define the discriminator model
def build_discriminator(img_shape):
    model = Sequential()

    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

# Define the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False

    model = Sequential()
    model.add(generator)
    model.add(discriminator)

    return model

# Hyperparameters
latent_dim = 100
img_shape = (28, 28, 1)  # Replace with the shape of your face images
epochs = 10000
batch_size = 64

# Build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build and compile the generator
generator = build_generator(latent_dim)
discriminator.trainable = False  # Prevent discriminator updates during GAN training
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Load and preprocess your face dataset (you should replace this with your dataset)
# Example using MNIST dataset for simplicity
(X_train, _), (_, _) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5  # Normalize images to [-1, 1]
X_train = np.expand_dims(X_train, axis=-1)




import os
import matplotlib.pyplot as plt

def save_generated_images(epoch, generator, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    generated_images = generator.predict(noise)

    # Rescale generated images to [0, 1]
    generated_images = 0.5 * generated_images + 0.5

    fig, axs = plt.subplots(dim[0], dim[1], figsize=figsize)
    for i in range(examples):
        axs[i].imshow(generated_images[i, :, :, 0], cmap='gray')
        axs[i].axis('off')

    plt.tight_layout()
    if not os.path.exists("generated_images"):
        os.makedirs("generated_images")
    plt.savefig("generated_images/epoch_%d.png" % epoch)
    plt.close()

    
    
    
# Training loop
for epoch in range(epochs):

    # Train discriminator
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_imgs = X_train[idx]
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_imgs = generator.predict(noise)
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))
    d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print progress
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

    # Save generated images at specific intervals
    if epoch % 1000 == 0:
        save_generated_images(epoch, generator)


2/2 [==============================] - 0s 70ms/step
Epoch 0, D Loss: 0.698899894952774, G Loss: 0.6878691911697388
2/2 [==============================] - 0s 63ms/step
Epoch 100, D Loss: 0.0039467013702960685, G Loss: 4.90062141418457
2/2 [==============================] - 0s 112ms/step


2/2 [==============================] - 0s 84ms/step
Epoch 200, D Loss: 0.0007640990224899724, G Loss: 6.603321552276611
2/2 [==============================] - 0s 90ms/step
Epoch 300, D Loss: 0.0003121940420669489, G Loss: 7.390738010406494
2/2 [==============================] - 0s 107ms/step
Epoch 400, D Loss: 0.00018384021313977428, G Loss: 8.105327606201172
2/2 [==============================] - 0s 169ms/step


2/2 [==============================] - 0s 111ms/step
Epoch 500, D Loss: 9.282272387167723e-05, G Loss: 8.598923683166504
2/2 [==============================] - 0s 67ms/step
Epoch 600, D Loss: 6.654977619291458e-05, G Loss: 8.946449279785156
2/2 [==============================] - 0s 159ms/step
Epoch 700, D Loss: 4.461523356269481e-05, G Loss: 9.33143424987793
2/2 [==============================] - 0s 113ms/step


2/2 [==============================] - 0s 54ms/step
Epoch 800, D Loss: 3.2588721211368465e-05, G Loss: 9.64532470703125
2/2 [==============================] - 0s 114ms/step
Epoch 900, D Loss: 2.732157776819122e-05, G Loss: 9.818642616271973
2/2 [==============================] - 0s 97ms/step
Epoch 1000, D Loss: 2.1450225858643535e-05, G Loss: 10.094703674316406
2/2 [==============================] - 0s 81ms/step


2/2 [==============================] - 0s 62ms/step
Epoch 1100, D Loss: 3.0091828969736056e-05, G Loss: 9.720942497253418
2/2 [==============================] - 0s 121ms/step
Epoch 1200, D Loss: 2.06393732411006e-05, G Loss: 10.097455978393555
2/2 [==============================] - 0s 95ms/step
Epoch 1300, D Loss: 1.5071036137515037e-05, G Loss: 10.411706924438477
2/2 [==============================] - 0s 70ms/step


2/2 [==============================] - 0s 103ms/step
Epoch 1400, D Loss: 1.1462177508470006e-05, G Loss: 10.685135841369629
2/2 [==============================] - 0s 69ms/step
Epoch 1500, D Loss: 8.97453863057951e-06, G Loss: 10.929618835449219
2/2 [==============================] - 0s 184ms/step


2/2 [==============================] - 0s 57ms/step
Epoch 1600, D Loss: 7.241724753179191e-06, G Loss: 11.143951416015625
2/2 [==============================] - 0s 52ms/step
Epoch 1700, D Loss: 5.884530828064935e-06, G Loss: 11.35137939453125
2/2 [==============================] - 0s 49ms/step
Epoch 1800, D Loss: 4.8466937591952475e-06, G Loss: 11.54533576965332
2/2 [==============================] - 0s 114ms/step


2/2 [==============================] - 0s 50ms/step
Epoch 1900, D Loss: 4.035626223855213e-06, G Loss: 11.728397369384766
2/2 [==============================] - 0s 56ms/step
Epoch 2000, D Loss: 3.3906041561997113e-06, G Loss: 11.90250015258789
2/2 [==============================] - 0s 107ms/step
Epoch 2100, D Loss: 2.870736682220737e-06, G Loss: 12.06889533996582
2/2 [==============================] - 0s 92ms/step


2/2 [==============================] - 0s 115ms/step
Epoch 2200, D Loss: 2.446275134460052e-06, G Loss: 12.228986740112305
1/2 [==============>...............] - ETA: 0s